In [1]:
%matplotlib inline
import utils; utils.importlib.reload(utils)
from utils import *

Using TensorFlow backend.


In [5]:
x = random((30,2))
y = np.dot(x, [1.,3.]) + 1

In [31]:
lm = Sequential([Dense(1, input_shape=(2,))])

In [32]:
lm.compile(optimizer=SGD(lr=0.1),loss='mse')

In [33]:
lm.evaluate(x,y)

30/30 [==============================] - 0s


10.702936172485352

In [34]:
lm.fit(x,y,nb_epoch=5,batch_size=1)

Epoch 1/50
30/30 [==============================] - 0s - loss: 1.1600     
Epoch 2/50
30/30 [==============================] - 0s - loss: 0.3885     
Epoch 3/50
30/30 [==============================] - 0s - loss: 0.1288     
Epoch 4/50
30/30 [==============================] - 0s - loss: 0.0473     
Epoch 5/50
30/30 [==============================] - 0s - loss: 0.0141     
Epoch 6/50
30/30 [==============================] - 0s - loss: 0.0066         
Epoch 7/50
30/30 [==============================] - 0s - loss: 0.0031     
Epoch 8/50
30/30 [==============================] - 0s - loss: 0.0014         
Epoch 9/50
30/30 [==============================] - 0s - loss: 7.2260e-04     
Epoch 10/50
30/30 [==============================] - 0s - loss: 4.0935e-04     
Epoch 11/50
30/30 [==============================] - 0s - loss: 2.1931e-04     
Epoch 12/50
30/30 [==============================] - 0s - loss: 1.3166e-04 
Epoch 13/50
30/30 [==============================] - 0s - loss: 7.9316e-05   

In [35]:
lm.evaluate(x,y)

30/30 [==============================] - 0s


2.3021585722829419e-13

In [36]:
lm.get_weights()

[array([[ 1.],
        [ 3.]], dtype=float32), array([ 1.], dtype=float32)]

In [108]:
path = 'data/dogscats/'
path = path + 'sample/'
model_path = path + 'models/'
if not os.path.lexists(model_path): os.mkdir(model_path)

In [109]:
batch_size=4

In [110]:
from vgg16 import Vgg16
vgg = Vgg16()

In [111]:
model = vgg.model

Get true labels for all images

Get 1000 imagenet category pred for each image

Make linear model mapping (1000,1)->(2,1) for imagenet predictions->cats/dogs

In [112]:
batches = get_batches(path + 'train', shuffle=False,batch_size=1)
val_batches = get_batches(path + 'valid',shuffle=False,batch_size=1)

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [113]:
def save_array(fname,arr): c=bcolz.carray(arr,rootdir=fname,mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [114]:
val_data = get_data(path+'valid')

Found 8 images belonging to 2 classes.


In [115]:
tr_data = get_data(path+'train')

Found 16 images belonging to 2 classes.


In [116]:
save_array(model_path + 'tr_data.bc',tr_data)
save_array(model_path + 'val_data.bc',val_data)

In [117]:
tr_data = load_array(model_path + 'tr_data.bc')
val_data = load_array(model_path + 'val_data.bc')

In [118]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [119]:
val_classes = val_batches.classes
tr_classes = batches.classes
val_labels = onehot(val_classes)
tr_labels = onehot(tr_classes)

In [120]:
tr_features = model.predict(tr_data,batch_size=batch_size)
val_features = model.predict(val_data,batch_size=batch_size)

In [121]:
save_array(model_path + 'tr_lastlayer_features.bc',tr_features)
save_array(model_path + 'val_lastlayer_features.bc',val_features)

In [122]:
lm = Sequential([Dense(2, activation='softmax',input_shape=(1000,))])

In [123]:
lm.compile(optimizer=RMSprop(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

In [124]:
batch_size = 4

In [125]:
lm.fit(tr_features,tr_labels,batch_size=batch_size,validation_data=(val_features,val_labels),nb_epoch=5)

Train on 16 samples, validate on 8 samples
Epoch 1/5
16/16 [==============================] - 0s - loss: 0.6938 - acc: 0.3750 - val_loss: 0.6679 - val_acc: 1.0000
Epoch 2/5
16/16 [==============================] - 0s - loss: 0.6661 - acc: 0.8125 - val_loss: 0.6481 - val_acc: 1.0000
Epoch 3/5
16/16 [==============================] - 0s - loss: 0.6471 - acc: 0.9375 - val_loss: 0.6314 - val_acc: 1.0000
Epoch 4/5
16/16 [==============================] - 0s - loss: 0.6292 - acc: 0.9375 - val_loss: 0.6169 - val_acc: 1.0000
Epoch 5/5
16/16 [==============================] - 0s - loss: 0.6167 - acc: 1.0000 - val_loss: 0.6036 - val_acc: 1.0000


In [126]:
from IPython.display import FileLink
FileLink('Tstlesson2.ipynb')

/home/jovyan/work/fastai/courses/deeplearning1/nbs/Tstlesson2.ipynb